In [6]:
import os
import random
from datetime import datetime

import numpy as np
import scipy as sp
import sklearn as sk
import torch
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import RandomizedSearchCV
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from dataset import TenBarsCantileverTrussSingleEADataset
from models.architecture import MultiLayerPerceptron
from models.processing import StandardScaler
import optuna


In [7]:
filepath = {
    'train': "data/dataset/10_bar_truss/train/data.hdf5",
    'validation': "data/dataset/10_bar_truss/validation/data.hdf5",
    'test': "data/dataset/10_bar_truss/test/data.hdf5"
}

In [8]:
N_EPOCH = 100
BATCH_SIZE = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

_train_dataset = TenBarsCantileverTrussSingleEADataset(filepath['train'])
_test_dataset = TenBarsCantileverTrussSingleEADataset(filepath['test'])
_validation_dataset = TenBarsCantileverTrussSingleEADataset(filepath['validation'])

train_dataloader = DataLoader(_train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=1, persistent_workers=True)
test_dataloader = DataLoader(_test_dataset, batch_size=BATCH_SIZE, shuffle=True,
                             num_workers=1, persistent_workers=True)
validation_dataloader = DataLoader(_validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
                                   num_workers=1, persistent_workers=True)

connectivity = torch.tensor([[0, 1, 3, 4, 1, 2, 0, 3, 1, 4],
                             [1, 2, 4, 5, 4, 5, 4, 1, 5, 2]]).T

support = torch.tensor([0, 1, 6, 7])

In [9]:
_train_dataset[0][0].shape

torch.Size([31])

In [19]:
scaler_input = StandardScaler(22).to(device)
scaler_target = StandardScaler(1).to(device)

for input, target, _, _, _ in train_dataloader:
    scaler_input.partial_fit(input.to(device))
    scaler_target.partial_fit(target.to(device))

# Regression with kPCA

In [29]:
x = []
y = []
for x_i, y_i, _, _, _ in torch.utils.data.ConcatDataset((_train_dataset, _test_dataset)):
    x.append(x_i.cpu().detach().numpy())
    y.append(y_i.cpu().detach().numpy())


In [31]:

from sklearn.linear_model import LinearRegression

pipe_1 = sk.pipeline.Pipeline([
    ('scaler', sk.preprocessing.StandardScaler()),
    ('kpca', KernelPCA()),
    ('estimator', LinearRegression())
])

model = TransformedTargetRegressor(regressor=pipe_1, transformer=StandardScaler())

TypeError: StandardScaler.__init__() missing 1 required positional argument: 'n_features'

In [ ]:
def conditional_param_distributions(n_iter=50, random_state=42):
    np.random.seed(random_state)
    param_list = []

    for _ in range(n_iter):
        # Randomly select a kernel
        kernel = random.choice(['rbf', 'poly', 'sigmoid', 'rbf', 'cosine'])

        # Base parameters
        params = {
            'regressor__kpca__kernel': kernel,
            'regressor__kpca__n_components': np.random.randint(5, 20),  # n_components between 5 and 10
        }

        # Conditional parameters based on kernel type
        if kernel in ['poly', 'rbf', 'sigmoid']:
            params['regressor__kpca__gamma'] = sp.stats.uniform(0, 5)  # gamma in range [0, 5]
        if kernel in ['poly']:
            params['regressor__kpca__degree'] = random.randint(1, 6)  # degree: 1, 2, 3, 4, 5
        if kernel in ['poly', 'sigmoid']:
            params['regressor__kpca__coef0'] = sp.stats.uniform(0, 5)  # coef0 in range [0, 5]

        # Append to list
        param_list.append(params)

    return param_list


# Generate conditional parameter combinations
param_distributions = conditional_param_distributions(n_iter=50)

# RandomizedSearchCV with custom parameter sampling
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='r2',
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# No PINN

In [5]:
layers = [20, 20, 20]
model = MultiLayerPerceptron(22, layers, 1).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

fn_loss = nn.MSELoss().to(device)

In [8]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
log_dir = './_runs/Single_EA_prediction_{}'.format(timestamp)

if not os.path.exists(f"{log_dir}/log"):
    os.makedirs(f"{log_dir}/log")

if not os.path.exists(f"{log_dir}/weights"):
    os.makedirs(f"{log_dir}/weights")

writer = SummaryWriter(log_dir + "/log")

best_v_loss = np.inf

for epoch in range(N_EPOCH):
    print(f'Epoch {epoch + 1:3d}/{N_EPOCH}')

    running_loss = 0.
    last_loss = np.inf

    model.train(True)
    for i, data in enumerate(train_dataloader):
        inputs, target, _, _, _ = data
        inputs = inputs.to(device)
        target = target.to(device)
        inputs = scaler_input.transform(inputs)
        target = scaler_target.transform(target)

        optimizer.zero_grad()

        ea_pred = model(inputs)

        loss = fn_loss(ea_pred, target)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:
            last_loss = running_loss / 100

            tb_x = epoch * len(train_dataloader) + i + 1
            writer.add_scalar('Loss during training', last_loss, tb_x)

            running_loss = 0.
            running_physic_loss = 0.
            running_data_loss = 0.

            # print(f"    Batch {i:3d}/{len(train_dataloader):3d}: train loss {running_loss:.4e}")

    model.eval()
    with torch.no_grad():
        v_loss = 0
        for data in validation_dataloader:
            inputs, target, _, _, _ = data

            inputs = scaler_input.transform(inputs.to(device))
            target = scaler_target.transform(target.to(device))

            ea_pred = model(inputs)

            v_loss += fn_loss(ea_pred, target).item()

    v_loss /= len(validation_dataloader)

    writer.add_scalars('Loss per epoch', {'Training loss': last_loss, 'Validation loss': v_loss, }, epoch + 1)
    writer.flush()

    print(f'Epoch {epoch + 1}: Train Loss: {last_loss:.4e} - Validation Loss: {v_loss:.4e}')

    if v_loss < best_v_loss:
        best_vloss = v_loss
        model_path = f"{log_dir}/weights/model_{timestamp}_{epoch}"
        torch.save(model.state_dict(), model_path)


Epoch   1/100
Epoch 1: Train Loss: 1.8448e-01 - Validation Loss: 1.7629e-01
Epoch   2/100
Epoch 2: Train Loss: 1.4096e-01 - Validation Loss: 1.3043e-01
Epoch   3/100
Epoch 3: Train Loss: 1.3603e-01 - Validation Loss: 1.2186e-01
Epoch   4/100
Epoch 4: Train Loss: 1.1200e-01 - Validation Loss: 1.0986e-01
Epoch   5/100
Epoch 5: Train Loss: 1.0099e-01 - Validation Loss: 9.8786e-02
Epoch   6/100
Epoch 6: Train Loss: 9.6123e-02 - Validation Loss: 9.0748e-02
Epoch   7/100
Epoch 7: Train Loss: 9.1241e-02 - Validation Loss: 9.0887e-02
Epoch   8/100
Epoch 8: Train Loss: 8.0505e-02 - Validation Loss: 8.1564e-02
Epoch   9/100
Epoch 9: Train Loss: 7.8884e-02 - Validation Loss: 7.5684e-02
Epoch  10/100
Epoch 10: Train Loss: 7.6216e-02 - Validation Loss: 7.7424e-02
Epoch  11/100
Epoch 11: Train Loss: 7.1060e-02 - Validation Loss: 7.0331e-02
Epoch  12/100
Epoch 12: Train Loss: 6.5600e-02 - Validation Loss: 6.4784e-02
Epoch  13/100


KeyboardInterrupt: 

In [7]:
model.eval()
acc = 0.
for inputs, target, _, _, _ in test_dataloader:
    inputs = scaler_input.transform(inputs)
    pred = model(inputs)
    pred = scaler_target.inverse_transform(pred)
    break

In [9]:
target

tensor([[1.2970e+08],
        [1.3646e+08],
        [2.0592e+08],
        [7.0743e+07],
        [1.2638e+08],
        [3.5457e+08],
        [1.0291e+08],
        [2.4766e+08],
        [3.0544e+08],
        [2.9172e+08],
        [3.1650e+08],
        [3.1341e+08],
        [2.0217e+08],
        [2.9202e+08],
        [1.7945e+08],
        [6.3652e+07],
        [1.8856e+08],
        [9.2852e+07],
        [3.7170e+08],
        [2.2786e+08],
        [1.5425e+08],
        [2.9590e+08],
        [1.1574e+08],
        [1.4083e+08],
        [9.8264e+07],
        [8.1517e+07],
        [7.9395e+07],
        [1.3565e+08],
        [3.3740e+08],
        [2.9014e+08],
        [3.3871e+08],
        [3.0739e+08],
        [2.9872e+08],
        [2.0169e+08],
        [2.7607e+08],
        [9.2383e+07],
        [1.8717e+08],
        [1.7665e+08],
        [1.3416e+08],
        [7.2358e+07],
        [1.8986e+08],
        [1.6908e+08],
        [2.4954e+08],
        [2.0343e+08],
        [2.3525e+08],
        [3

In [26]:
target[0:2]

tensor([[40765909.4608],
        [94472455.9233]], dtype=torch.float64)